<a href="https://colab.research.google.com/github/Zedbog/First_repository/blob/master/zadania_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Inizcalizacja środowiska Google Colab
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 51kB 1.6MB/s 


In [14]:
# zaimporotowanie bibliotek sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [4]:
engine = create_engine('mysql+pymysql://root:root@/')

In [ ]:
engine.execute('CREATE DATABASE cinematic')
engine.execute('USE cinematic')

In [27]:
eng = create_engine("mysql+pymysql://root:root@/cinematic")
base = declarative_base()

class Directors(base):
  __tablename__ = 'directors'

  director_id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String(30), nullable=False)
  surname = Column(String(30), nullable=False)
  rating = Column(Integer, nullable=False)

  def __repr__(self):
    return f"<Director: id: {self.director_id}, Name: {self.name}, Surname: {self.surname}, Rating: {self.rating}>"

class Movies(base):
  __tablename__ = 'movies'

  movie_id = Column(Integer, primary_key=True, autoincrement=True)
  title = Column(String(60), nullable=False)
  year = Column(Integer, nullable=False)
  category = Column(String(30), nullable=False)
  director_id = Column(Integer, nullable=False)
  rating = Column(Integer, nullable=False)

  def __repr__(self):
    return f"<Movie: id: {self.movie_id}, Title: {self.title}, Year: {self.year}, Category: {self.category}, Director_id: {self.director_id}, Rating: {self.rating}>"

In [28]:
#Utworzenie sesji do wykonywania zapytań
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=eng)
session = Session()

In [ ]:
directors = [{'name': 'Frank', 'surname': 'Darabont', 'rating': 7}, {'name':
'Francis Ford', 'surname': 'Coppola', 'rating': 8}, {'name': 'Quentin',
'surname': 'Tarantino', 'rating': 10}, {'name': 'Christopher', 'surname':
'Nolan', 'rating': 9}, {'name': 'David', 'surname': 'Fincher', 'rating': 7}]
movies = [{'title': 'The Shawshank Redemption', 'year': 1994,
'category': 'Drama', 'director_id': 1, 'rating': 8}, {'title': 'The Green Mile',
'year': 1999, 'category': 'Drama', 'director_id': 1, 'rating': 6}, {'title': 'The Godfather', 'year': 1972, 'category': 'Crime', 'director_id': 2, 'rating': 7},
{'title': 'The Godfather III', 'year': 1990, 'category': 'Crime', 'director_id':
2, 'rating': 6}, {'title': 'Pulp Fiction', 'year': 1994, 'category': 'Crime',
'director_id': 3, 'rating': 9}, {'title': 'Inglourious Basterds', 'year': 2009,
'category': 'War', 'director_id': 3, 'rating': 8}, {'title': 'The Dark Knight',
'year': 2008, 'category': 'Action', 'director_id': 4, 'rating': 9}, {'title':
'Interstellar', 'year': 2014, 'category': 'Sci-fi', 'director_id': 4, 'rating': 8},
{'title': 'The Prestige', 'year': 2006, 'category': 'Drama', 'director_id': 4,
'rating': 10}, {'title': 'Fight Club', 'year': 1999, 'category': 'Drama',
'director_id': 5, 'rating': 7}, {'title': 'Zodiac', 'year': 2007, 'category':
'Crime', 'director_id': 5, 'rating': 5}, {'title': 'Seven', 'year': 1995,
'category': 'Drama', 'director_id': 5, 'rating': 8}, {'title': 'Alien 3', 'year':
1992, 'category': 'Horror', 'director_id': 5, 'rating': 5}]

In [32]:
def insert_data(session, base, params):
  session.add(base(**params))
  session.commit()

In [33]:
for director in directors:
  insert_data(session, Directors, director)
for movie in movies:
  insert_data(session, Movies, movie)

ProgrammingError: ignored

In [31]:
for director in session.query(Directors).all():
  print(director) 

ProgrammingError: ignored

In [29]:
#Wypisanie wszysstkich filmów z kategorii Drama
for movie in session.query(Movies).filter(Movies.category == 'Drama'):
  print(movie) 

ProgrammingError: ignored